In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imshow
import os
import cv2

from PIL import Image
from resizeimage import resizeimage


In [ ]:
#for kernel optimization
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='True'

In [ ]:
#loading training images from the dataset

def load_images_from_folder(mod,folder):
    for filename in os.listdir(mod):
        if any([filename.endswith(x) for x in ['.jpeg', '.jpg']]):
            img = cv2.imread(os.path.join(mod, filename), 0)
            #img = cv2.imread(os.path.join(mod, filename), as_gray=True)
            res = mod.rsplit('/', 1)[1]
            #appending the training labels
            train_labels.append(int(res))
            resized = cv2.resize(img,(128,128), interpolation = cv2.INTER_AREA)
            #appending training images
            train_images.append(np.array(resized))

#digit + operator folders
folders = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13']
folder_dir="./Dataset modified/"

train_labels=[]
train_images=[]
i=0
for folder in os.listdir(folder_dir):
    mod=folder_dir+folders[i]
    print(mod)
    i+=1
    load_images_from_folder(mod,folder)


In [ ]:
#loading testing images from the dataset

def load_images_from_folder_testing(mod,folder):
    images = []
    for filename in os.listdir(mod):
        if any([filename.endswith(x) for x in ['.jpeg', '.jpg']]):
            img = cv2.imread(os.path.join(mod, filename), 0)
            #img = cv2.imread(os.path.join(mod, filename), as_gray=True)
            res = mod.rsplit('/', 1)[1]
            #appending the testing labels
            test_labels.append(int(res))
            resized = cv2.resize(img,(128,128), interpolation = cv2.INTER_AREA)
            #appending the training labels
            test_images.append(np.array(resized))
    return images

#digit + operator folders
folders = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13']
folder_dir="./Dataset modified test/"

test_labels=[]
test_images=[]
i=0
for folder in os.listdir(folder_dir):
    mod=folder_dir+folders[i]
    print(mod)
    i+=1
    images = load_images_from_folder_testing(mod,folder)


In [ ]:
#changing the pixel intensity value to uint8
train_labels=np.asarray(train_labels).astype('uint8')
test_labels=np.asarray(test_labels).astype('uint8')
train_images = np.array(train_images)
import numpy as np
test_images = np.array(test_images)


print("train images before flattening",train_images.shape)
print("test images before flattening",test_images.shape)

# flatten images
train_images = train_images.reshape((train_images.shape[0], -1))
test_images = test_images.reshape((test_images.shape[0], -1))

print("train images",train_images.shape)
print("train labels",train_labels.shape)

print("test images",test_images.shape)
print("test labels",test_labels.shape)

In [ ]:
from urllib.request import urlretrieve

import nengo
import nengo_dl
import tensorflow as tf


In [ ]:
#MODEL
with nengo.Network(seed=0) as net:
    # set some default parameters for the neurons that will make
    # the training progress more smoothly
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([100])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None
    neuron_type = nengo.LIF(amplitude=0.01)

    # this is an optimization to improve the training speed,
    # since we won't require stateful behaviour in this example
    nengo_dl.configure_settings(stateful=False)

    # the input node that will be used to feed in input images
    inp = nengo.Node(np.zeros(128 * 128))

    # add the first convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(
        filters=32, kernel_size=3))(inp, shape_in=(128,128, 1))
    x = nengo_dl.Layer(neuron_type)(x)

    # add the second convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(
        filters=64, strides=2, kernel_size=3))(x, shape_in=(126, 126, 32))
    x = nengo_dl.Layer(neuron_type)(x)

    # add the third convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(
        filters=128, strides=2, kernel_size=3))(x, shape_in=(62,62, 64))
    x = nengo_dl.Layer(neuron_type)(x)
    
    # add the fourth convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(
        filters=256, strides=2, kernel_size=3))(x, shape_in=(30,30, 128))
    x = nengo_dl.Layer(neuron_type)(x)
    
    
    # add the fifth convolutional layer
    x = nengo_dl.Layer(tf.keras.layers.Conv2D(
        filters=512, strides=2, kernel_size=3))(x, shape_in=(14,14, 256))
    x = nengo_dl.Layer(neuron_type)(x)

    # linear readout
    out = nengo_dl.Layer(tf.keras.layers.Dense(units=14))(x)

    # we'll create two different output probes, one with a filter
    # (for when we're simulating the network over time and
    # accumulating spikes), and one without (for when we're
    # training the network using a rate-based approximation)
    # probes essentially collect data over time
    out_p = nengo.Probe(out, label="out_p")
    out_p_filt = nengo.Probe(out, synapse=0.1, label="out_p_filt")

# create the simulator that will run the CSNN
minibatch_size = 100
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size, device="/gpu:0")

# add single timestep to training data
train_images = train_images[:, None, :]
train_labels = train_labels[:, None, None]

# when testing our network with spiking neurons we will need to run it
# over time, so we repeat the input/target data for a number of
# timesteps.


n_steps = 30
test_images = np.tile(test_images[:, None, :], (1, n_steps, 1))
test_labels = np.tile(test_labels[:, None, None], (1, n_steps, 1))

# we create a custom accuracy classification problem because we are only concerned with the accuracy of the actual SNN
def classification_accuracy(y_true, y_pred):
    return tf.metrics.sparse_categorical_accuracy(
        y_true[:, -1], y_pred[:, -1])

# note that we use `out_p_filt` when testing (to reduce the spike noise)
sim.compile(loss={out_p_filt: classification_accuracy})
print("accuracy before training:",
      sim.evaluate(test_images, {out_p_filt: test_labels}, verbose=0)["loss"])
   
# train the rate-based approximation of the CSNN
do_training = True
if do_training:
    # run training
    sim.compile(
        optimizer=tf.optimizers.RMSprop(0.001),
        #optimizer=tf.optimizers.Adam(0.001, 0.9, 0.999),
        loss={out_p: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
        metrics={out_p: classification_accuracy}
    )
    sim.fit(train_images, {out_p: train_labels}, epochs=20, verbose=1)
    
    # save the parameters to file
    sim.save_params("./mnist_params")
else:
    # download pretrained weights
    urlretrieve(
        "https://drive.google.com/uc?export=download&"
        "id=1l5aivQljFoXzPP5JVccdFXbOYRv3BCJR",
        "mnist_params.npz")

    # load parameters
    sim.load_params("./mnist_params")
    
#print the accuracy of the CSNN
sim.compile(loss={out_p_filt: classification_accuracy})
print("accuracy after training:",
      sim.evaluate(test_images, {out_p_filt: test_labels}, verbose=0)["loss"])
      
# code to plot example outputs from the CSNN
data = sim.predict(test_images[:854])
sim.close()


for i in range(5):
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(test_images[i, 0].reshape((128, 128)), cmap="gray")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.plot(tf.nn.softmax(data[out_p_filt][i]))
    plt.legend([str(i) for i in range(15)], loc="upper left")
    plt.xlabel("timesteps")
    plt.ylabel("probability")
    plt.tight_layout()


In [ ]:
for i in range(800,810):
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(test_images[i, 0].reshape((128, 128)), cmap="gray")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.plot(tf.nn.softmax(data[out_p_filt][i]))
    plt.legend([str(i) for i in range(15)], loc="upper left")
    plt.xlabel("timesteps")
    plt.ylabel("probability")
    plt.savefig(str(test_labels[i][0])+str(i)+".jpg")
    plt.tight_layout()
